In [ ]:
#Name: Anamika Chatterjee
#Student ID: 202004102
#Email: x2020dxz@stfx.ca

In [2]:
#Loading the dataset
import pandas as pd
Train = pd.read_csv("train.csv")
Train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#Looking for null values in the dataset
Train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
# Filling in null values in age column with the mean value 
Train['Age'].fillna(Train['Age'].mean(),inplace=True)

In [3]:
mode=Train['Embarked'].mode()
mode

0    S
dtype: object

In [5]:
#filling in the null value in the embarked column with 'S' as is seems to be the most frequently occuring
Train['Embarked'].fillna('S', inplace=True)

In [7]:
#droping columns that are inconsequential features
Train.drop(['PassengerId', 'Name','Ticket','Cabin'], axis=1, inplace=True )

In [8]:
#coverting 'embarked' and 'sex' categorical values into numerical value
Train['Embarked'].replace(['S','C','Q'], [1,2,3], inplace=True)
Train['Sex'].replace(['male','female'], [0,1], inplace=True)
Train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,1
1,1,1,1,38.0,1,0,71.2833,2
2,1,3,1,26.0,0,0,7.9250,1
3,1,1,1,35.0,1,0,53.1000,1
4,0,3,0,35.0,0,0,8.0500,1


In [9]:
# normalizing 'age' and 'fare' values
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
s=['Age','Fare']
Train[s] = ss.fit_transform(Train[s])
Train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,-0.592481,1,0,-0.502445,1
1,1,1,1,0.638789,1,0,0.786845,2
2,1,3,1,-0.284663,0,0,-0.488854,1
3,1,1,1,0.407926,1,0,0.420730,1
4,0,3,0,0.407926,0,0,-0.486337,1


In [11]:
X=Train.drop(['Survived'],axis=1)
y=Train['Survived']

In [12]:
#defining parameters for two models that need to be compared
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

model_param = {
    'RandomForestClassifier':{
        'model':RandomForestClassifier(random_state=1),
        'param':{
            'n_estimators': [50,100,250],
            'max_depth':[4,5],
            'criterion':['entropy']
        }
    },
    'GradientBoostingClassifier':{
        'model':GradientBoostingClassifier(random_state=1),
        'param':{
           "max_depth": [8,9],
            "n_estimators": [100,300]
        }
    }
}


In [13]:
score=[]
for model_name, mp in model_param.items():
    mod_compare = GridSearchCV(estimator=mp['model'],param_grid=mp['param'],cv=5,return_train_score=False)
    mod_compare.fit(X,y)
    score.append({
        'model': model_name,
        'best_score': mod_compare.best_score_,
        'best_params': mod_compare.best_params_
    })

In [14]:
mod_score_compare=pd.DataFrame(score,columns=['model','best_score','best_params'])
mod_score_compare

,model,best_score,best_params
0,RandomForestClassifier,0.822679,"{'criterion': 'entropy', 'max_depth': 4, 'n_es..."
1,GradientBoostingClassifier,0.804758,"{'max_depth': 9, 'n_estimators': 100}"


In [15]:
#loading test data
Test_df= pd.read_csv("test.csv")
test= pd.read_csv("test.csv")
Test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [16]:
# filling 'age' and 'fare' null values with mean values
Test_df['Age'].fillna(Test_df['Age'].mean(),inplace=True)
Test_df['Fare'].fillna(Test_df['Fare'].mean(),inplace=True)

In [17]:
Test_df['Embarked'].replace(['S','C','Q'], [1,2,3], inplace=True)
Test_df['Sex'].replace(['male','female'], [0,1], inplace=True)
Test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,3
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,1
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,3
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,1


In [18]:
Test_df.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True )

In [19]:
s=['Age','Fare']
Test_df[s] = ss.fit_transform(Test_df[s])

In [20]:
rfc_mod=RandomForestClassifier(max_depth=4,random_state=1,n_estimators= 250,criterion='entropy')
rfc_mod.fit(X,y)

RandomForestClassifier(criterion='entropy', max_depth=4, n_estimators=250,
                       random_state=1)

In [21]:
prediction=rfc_mod.predict(Test_df)
Submission_1 =pd.DataFrame({"PassengerId": test['PassengerId'], "Survived": prediction})
Submission_1.to_csv('F:\Anamika\Data Minning\Submission_titanic.csv', index=False)